# IMPORT LIBRARIES

In [55]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
import os

# HELPER FUNCTIONS

In [47]:
# FUNCTION TO OPEN FILES
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

# FUNCTION TO FORMAT CHAT HISTORY
def format_history(chat_history):
    return "\n".join(f"FRAGE: {turn['FRAGE']}\nANTWORT: {turn['ANTWORT']}" for turn in chat_history)

# CREDENTIALS

In [48]:
# DEFINE CREDENTIALS
groq_api_key = open_file("../keys/groq_api.txt")
openai_api_key = open_file("../keys/openaiapikey.txt")

# LOAD CONTEXT DOCUMENTS FOR RAG

In [49]:
# LOAD ALL FILES FROM FOLDER AND SAVE THEM IN A LIST (documents)
def load_and_process_txts(txt_folder_path):
    documents = []
    for file in os.listdir(txt_folder_path):
        if file.endswith('.txt'):
            txt_path = os.path.join(txt_folder_path, file)
            loader = TextLoader(txt_path)
            documents.extend(loader.load())
    return documents

documents = load_and_process_txts('../txt/')

# CREATE INDEX AND RETRIEVER

In [50]:
# FUNCTION TO CREATE INDEX OF DOCUMENTS AND RETRIEVER TO GET THEM
def initialize_vectorstore(documents, embeddings):
    return FAISS.from_documents(documents, embeddings)

vectorstore = initialize_vectorstore(documents, OpenAIEmbeddings(openai_api_key=openai_api_key))
retriever = vectorstore.as_retriever()

# CREATE PROMPT

In [51]:
# CREATE PROMPT TEMPLATE WITH THREE PARAMETERS (History, Context, Question)
template = """
Beantworte die Frage mit Hilfe des nachfolgenden Kontext und der bisherigen Konversation. 
Wenn du die Frage mit Hilfe des Kontext oder der bisherigen Konversation nicht beantworten kannst, 
antworte mit: "Das weiß ich leider nicht."

BISHERIGE KONVERSATION:
{history}

KONTEXT: {context}

FRAGE: {question}
"""

# CREATE PROMPT WITH TEMPLATE
prompt = ChatPromptTemplate.from_template(template)

# CREATE CHAIN

In [52]:
# CREATE PARSER
parser = StrOutputParser()

# VARIABLE TO MAINTAIN CONVERSATION HISTORY
chat_history = []

# BUILD CHAIN WITH SELECTED MODEL AND HISTORY
def build_chain(model_name, chat_history):
    
    # CHOOSE MODEL
    if model_name.startswith("gpt"):
        model = ChatOpenAI(api_key=openai_api_key, model=model_name)
    elif model_name.startswith("llama"):
        model = ChatGroq(model_name=model_name, temperature=0, groq_api_key=groq_api_key)
        #model = Ollama(model="llama3")
    else:
        model = ChatGroq(model_name=model_name, temperature=0, groq_api_key=groq_api_key)
    
    # BUILD THE CHAIN WITH THESES STEPS
    # 1: Use Prompt that has two Parameters: History and Question
    # 2: Pass Prompt to selected Model
    # 3: Pass Response to OutputParser to create a String
    chain = prompt | model | parser
    
    return chain

# INVOKE CHAIN

In [53]:
# FUNCTION TO INVOKE CHAIN WITH QUESTION AND SELECTED MODEL
def get_answer(question, model_name):
    
    global chat_history
    
    # Build chain based on the selected model and using the chat history
    chain = build_chain(model_name, chat_history)
    
    # Get relevant Context to Question
    context = retriever.invoke(question)
    
    # Format chat history
    formatted_history = format_history(chat_history)
    
    # Get response for Question, using Context and History
    inputs = {
        "context": context,
        "history": formatted_history,
        "question": question
    }
    
    # Invoke the Chain using above Inputs
    response = chain.invoke(inputs)
    
    # Update chat history
    chat_history.append({"FRAGE": question, "ANTWORT": response})
    
    return response

In [58]:
print(get_answer("Was sind Kennzahlen linearer Regression?", "llama3"))

I see that you're asking a question about dimensionality reduction, specifically about the Principal Component Analysis (PCA) method. However, your question doesn't seem to match with the provided text.

To answer your question, PCA is a statistical method used for dimensionality reduction. It's not related to linear regression. If you're looking for information on linear regression, I can provide some general insights:

Linear regression is a supervised learning algorithm that predicts a continuous output variable based on one or more input features. The key performance metrics of linear regression include:

1. Mean Squared Error (MSE): This measures the average squared difference between predicted and actual values.
2. Coefficient of Determination (R-squared): This indicates how well the model fits the data, ranging from 0 (worst fit) to 1 (best fit).
3. Root Mean Squared Error (RMSE): This is the square root of MSE, giving an idea of the average distance between predicted and actual

In [42]:
print(get_answer("Kannst du mir die letzte Kennzahl genauer erklären bitte?", "gpt-4o"))

Ja, gerne! Die letzte Kennzahl, die in der bisherigen Konversation erwähnt wurde, ist der **Coefficient of Determination (R²)**. Lassen Sie mich diese Kennzahl genauer erläutern:

### Coefficient of Determination (R²)

Der Coefficient of Determination, auch bekannt als R²-Wert, ist eine statistische Kennzahl, die die Güte der Anpassung eines Regressionsmodells an die Daten beschreibt. Er gibt an, wie viel der Variabilität der abhängigen Variable (Y) durch die unabhängigen Variablen (X) im Modell erklärt wird.

#### Interpretation des R²-Werts:
- **R² = 1**: Das Modell erklärt die gesamte Variabilität in den Daten. Dies bedeutet, dass die vorhergesagten Werte perfekt mit den tatsächlichen Werten übereinstimmen.
- **R² = 0**: Das Modell erklärt keine der Variabilität in den Daten. Dies bedeutet, dass das Modell nicht besser ist als das naivste Modell, das einfach den Durchschnitt der tatsächlichen Werte vorhersagt.
- **0 < R² < 1**: Der Wert gibt den Anteil der erklärten Variabilität an.

In [44]:
print(get_answer("Super, was sind die Inhalte des Kurses?", "gpt-4o"))

Der Kurs "BUSINESS DATA ANALYST*in (IHK)" behandelt eine Vielzahl von Themen, die in den folgenden Kapiteln gegliedert sind:

### I. Grundlagen des Machine Learning
1. **Einführung in verantwortungsvolles Arbeiten mit Daten und Machine Learning**
   - Die Realität von Machine Learning
   - Fallstudie: Der COMPAS-Algorithmus
   - Zieldefinition und Qualitätsmessung in Machine Learning
   - Erfolgskriterien und Diskriminierung in Machine Learning
2. **Grundlagen der Statistik für Machine Learning**
   - Einführungen in statistische Grundlagen
   - Statistische Maßzahlen
   - Inferenzstatistik
   - Unterschied zwischen Korrelation und Kausalität
   - Datenvisualisierung
3. **Data Preprocessing – Der unsichtbare Held der Datenanalyse**
   - Methoden des Machine-Learning
   - Datenarten des Machine-Learning
   - Herausforderungen der Daten-Vorverarbeitung
   - Splitten von Daten für effektives Machine Learning
4. **Regression – Vorhersage und Verständnis von Zusammenhängen**
   - Einführung

In [45]:
print(get_answer("Kannst du mir Details zu den Inhalten von 7. geben?", "gpt-4o"))

Ja, gerne! Kapitel 7 des Kurses "BUSINESS DATA ANALYST*in (IHK)" behandelt den Umgang mit **Imbalanced Data**. Hier sind die Details zu diesem Kapitel:

### 7. Umgang mit Imbalanced Data
1. **Einführung in Imbalanced Data**
   - **Definition und Herausforderungen**: In diesem Abschnitt wird erklärt, was Imbalanced Data ist und welche Herausforderungen sie für Machine Learning Modelle darstellt. Imbalanced Data bedeutet, dass die Klassenverteilung in einem Datensatz stark unausgeglichen ist, z.B. wenn es viel mehr negative als positive Beispiele gibt.

2. **Methoden zur Behandlung von Imbalanced Data**
   - **Resampling Techniken**: Hier werden Techniken wie **Oversampling** und **Undersampling** vorgestellt, die dazu dienen, die Klassenverteilung auszugleichen.
   - **Resampling Beispiel**: Ein praktisches Beispiel, das zeigt, wie Resampling Techniken implementiert werden können, um die Balance in den Daten wiederherzustellen.
   - **Synthetische Datengenerierung**: Einführung in Metho